Create bulk figures to search for the best cloud-free image in each year for our validation glaciers

In [1]:
import rasterio as rio
import numpy as np
import os
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import geopandas as gpd
import gc
from rasterio.mask import mask

# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_images = os.path.join('C:',os.sep,'Users','lzell','Documents','Validation Imagery')


In [2]:
# formatting each image for display. they will initially come in as [ (B,G,R,N), :, : ]
band_r = 3
band_g = 2
band_b = 1

def format_LS(array):
    array = np.dstack( [array[band_r], array[band_g], array[band_b]] )
    return np.clip((array / 45000), 0, 1)

def format_S2(array):
    array = np.dstack( [array[band_r], array[band_g], array[band_b]] )
    return np.clip((array / 10000), 0, 1)

# define folder paths
S2_folder = os.path.join(folder_images, 'S2_all_validation_images')
L9_folder = os.path.join(folder_images, 'L9_all_validation_images')
L8_folder = os.path.join(folder_images, 'L8_all_validation_images')
L7_folder = os.path.join(folder_images, 'L7_all_validation_images')
L5_folder = os.path.join(folder_images, 'L5_all_validation_images')
output_folder = os.path.join(folder_AGVA, 'Validation', 'all_validation_search')

### Sentinel imagery here

In [6]:
### go through and make S2 imagery into figures

# get list of all S2 images we have (for all rgis)
all_S2_images = os.listdir(S2_folder)
print(all_S2_images[0])

# get list of unique rgis
all_S2_rgis = list(set( [i[:14] for i in all_S2_images] ))
print(all_S2_rgis)

# now for each unique rgi, go through to make images of it
for i in range(len(all_S2_rgis)):
    
    rgi_i = all_S2_rgis[i]

    # skip taku for now
    if rgi_i != "RGI60_01.08989": continue  # 'RGI60_01.01104', 'RGI60_01.08989', 'RGI60_01.00570', 'RGI60_01.09162', 'RGI60_01.01390'
#     if rgi_i == 'RGI60_01.01390': continue

    # get list of the images we have for this rgi
    rgi_image_names = [i for i in all_S2_images if i[:14]==rgi_i]
    
    # get date from all of these
    rgi_image_dates = [i[20:28] for i in rgi_image_names]
    rgi_image_dates = [f"{str(i)[:4]}-{str(i)[4:6]}-{str(i)[6:8]}" for i in rgi_image_dates]

    # get year from each
    rgi_image_years = [int(i[:4]) for i in rgi_image_dates]
    
    # make into df
    rgi_df = pd.DataFrame({'year':rgi_image_years, 'date':rgi_image_dates, 'path':rgi_image_names})
    rgi_df = rgi_df.sort_values('date')
    
    # for each year, make different images
    save=1
    for y in [2018,2019,2020,2021,2022]:
#         if y>2018: continue

        rgi_df_subset = rgi_df[rgi_df['year']==y]
        if len(rgi_df_subset)<1: continue

        rgi_df_subset = rgi_df_subset.sort_values('date')

        # start the image. have 4x4 inch square for each image
        # 2 columns, as many rows as needed
        ncol = 2
        nrow = max( len(rgi_df_subset)//ncol + 1, 2 )
        width = 8
        height = nrow*4
        
        if height>160: # figures cant be more than 160 inches tall, so we have to set dpi to smaller in this case
            dpi = int(65000/height)
        else:
            dpi=400
        
        print(f"{i} {rgi_i} {y}, {nrow} rows, {ncol} cols, {len(rgi_df_subset)} images")
        fig, axs = plt.subplots(nrow, ncol, figsize=(width,height), dpi=dpi)

        # iterate through subset df, opening each image and placing it on the output ifgure
        c=0
        for idx, row in rgi_df_subset.iterrows():
            # if c>5: continue 

            ax = axs[c//ncol, c%ncol]
            c+=1

            # define image path, open it
            image_filepath = os.path.join(S2_folder, row['path'])
            image = rio.open(image_filepath).read()
            # print(image.shape)

            # format for display
            image = format_S2(image)
            # print(image_display.shape)

            # add to figure
            ax.imshow(image, interpolation='nearest')

            # add date to figure
            ax.set_title(row['date'], size=6, pad=0)
        
        # final edits
        for ax in axs:
            for a in ax:
                a.axis('off')
        plt.tight_layout()

        # save as png
        if save:
            save_path = os.path.join(output_folder, f"{rgi_i}_S2_{y}.png")
            fig.savefig(save_path, dpi=dpi, bbox_inches='tight', pad_inches=0.05, transparent=False)
        
        # close figure, clear variables
        plt.close()
        del image
        # del image_display
        del fig, axs, ax, a
        del row, rgi_df_subset, idx
        gc.collect()
gc.collect()



RGI60_01.00570_GS2A_20180706T211511_015870_N02.tif
['RGI60_01.01390', 'RGI60_01.01104', 'RGI60_01.09162', 'RGI60_01.00570', 'RGI60_01.08989']
4 RGI60_01.08989 2018, 43 rows, 2 cols, 84 images
4 RGI60_01.08989 2019, 45 rows, 2 cols, 88 images
4 RGI60_01.08989 2020, 44 rows, 2 cols, 87 images
4 RGI60_01.08989 2021, 44 rows, 2 cols, 87 images
4 RGI60_01.08989 2022, 43 rows, 2 cols, 85 images


0

### Landsat imagery here

In [ ]:
### go through and make landsat imagery into figures

# get list of all landsat images we have (for all rgis)
all_L5_images = os.listdir(L5_folder)
all_L7_images = os.listdir(L7_folder)
all_L8_images = os.listdir(L8_folder)
all_L9_images = os.listdir(L9_folder)
all_LS_images = all_L5_images + all_L7_images + all_L8_images + all_L9_images
print(all_LS_images[0])

# get list of unique rgis
all_LS_rgis = list(set( [i[:14] for i in all_LS_images] ))
print(all_LS_rgis)

# now for each unique rgi, go through to make images of it
for i in range(len(all_LS_rgis)):
    # if i>0: continue
    # print(i)

    rgi_i = all_LS_rgis[i]

    # skip taku for now
    if rgi_i != "RGI60_01.08989": continue # 'RGI60_01.09162', 'RGI60_01.01104', 'RGI60_01.00570', 'RGI60_01.08989', 'RGI60_01.01390'
    # if rgi_i == 'RGI60_01.01390': continue

    # get list of the images we have for this rgi
    rgi_image_names = [i for i in all_LS_images if i[:14]==rgi_i]

    # get date from all of these
    rgi_image_dates = [i[32:40] for i in rgi_image_names]
    rgi_image_dates = [f"{str(i)[:4]}-{str(i)[4:6]}-{str(i)[6:8]}" for i in rgi_image_dates]
    # print(rgi_image_dates[0])

    # get year from each
    rgi_image_years = [int(i[:4]) for i in rgi_image_dates]

    # get satellite from each
    rgi_sats = [i[18] for i in rgi_image_names]

    # make into df
    rgi_df = pd.DataFrame({'year':rgi_image_years, 'date':rgi_image_dates, 'path':rgi_image_names, 'sat':rgi_sats})
    rgi_df = rgi_df.sort_values('date')
    # print(rgi_df.head())

    # for each year, make different images
    save=1
    for y in range(2000,2023):

        # if y>2017: continue
        # if y>2018: continue

        rgi_df_subset = rgi_df[rgi_df['year']==y]
        if len(rgi_df_subset)<1: continue

        rgi_df_subset = rgi_df_subset.sort_values('date')

        # start the image. have 4x4 inch square for each image
        # 2 columns, as many rows as needed
        ncol = 2
        nrow = max( len(rgi_df_subset)//ncol + 1, 2 )
        width = 8
        height = nrow*4

        print(f"{i} {rgi_i} {y}, {nrow} rows, {ncol} cols, {len(rgi_df_subset)} images")
        fig, axs = plt.subplots(nrow, ncol, figsize=(width,height), dpi=400, num=1)

        # iterate through subset df, opening each image and placing it on the output ifgure
        c=0
        for idx, row in rgi_df_subset.iterrows():
            # if c>5: continue

            ax = axs[c//ncol, c%ncol]
            c+=1

            # define image path, open it
            if row['sat']=='5': ls_folder = L5_folder
            if row['sat']=='7': ls_folder = L7_folder
            if row['sat']=='8': ls_folder = L8_folder

            image_filepath = os.path.join(ls_folder, row['path'])
            image = rio.open(image_filepath).read()
            # print(image.shape)

            # format for display
            image = format_LS(image)
            # print(image_display.shape)

            # add to figure
            ax.imshow(image, interpolation='nearest')

            # add date to figure
            ax.set_title(row['date'], size=6, pad=0)

        # final edits
        for ax in axs:
            for a in ax:
                a.axis('off')
        plt.tight_layout()

        # save as png
        if save:
            save_path = os.path.join(output_folder, f"{rgi_i}_LS_{y}.png")
            fig.savefig(save_path, dpi=400, bbox_inches='tight', pad_inches=0.05, transparent=False)

        # close figure, clear variables
        plt.close()
        del image
        # del image_display
        del fig, axs, ax, a
        del row, rgi_df_subset, idx
        gc.collect()
gc.collect()
